In [ ]:
# default_exp tree

# tree
> Tree

In [ ]:
# export
# from uid3.tree_node import TreeNode
from pyuid3.value import Value

In [ ]:
# export
class Condition:
    def __init__(self, att_name: str, value: Value, op='eq'):
        self.attName = attName
        self.value = value
        self.op = op